In [1]:
library(devtools)
devtools::install_github("prodriguezsosa/conText")

Loading required package: usethis




colorspace   (2.0-3      -> 2.1-0     ) [CRAN]
vctrs        (0.5.1      -> 0.5.2     ) [CRAN]
utf8         (1.2.2      -> 1.2.3     ) [CRAN]
fansi        (1.0.3      -> 1.0.4     ) [CRAN]
Rcpp         (1.0.9      -> 1.0.10    ) [CRAN]
RcppArmad... (0.11.4.3.1 -> 0.12.0.1.0) [CRAN]
RhpcBLASctl  (0.21-247.1 -> 0.23-42   ) [CRAN]
float        (0.3-0      -> 0.3-1     ) [CRAN]
data.table   (1.14.6     -> 1.14.8    ) [CRAN]
dplyr        (1.0.10     -> 1.1.0     ) [CRAN]
yaml         (2.3.6      -> 2.3.7     ) [CRAN]
ggplot2      (3.4.0      -> 3.4.1     ) [CRAN]
tidyr        (1.2.1      -> 1.3.0     ) [CRAN]


Installing 13 packages: colorspace, vctrs, utf8, fansi, Rcpp, RcppArmadillo, RhpcBLASctl, float, data.table, dplyr, yaml, ggplot2, tidyr

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in i.p(...):
“installation of package ‘vctrs’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘utf8’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘fansi’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘Rcpp’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘dplyr’ had non-zero exit status”
Warning message in i.p(...):
“installation of package ‘tidyr’ had non-zero exit status”


── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/Rtmp3ChBdg/remotese7c65911e/prodriguezsosa-conText-a3e1686/DESCRIPTION’ ... OK
* preparing ‘conText’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘conText_1.4.3.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
library(conText)
library(quanteda)
library(dplyr)
library(text2vec)
library(stringr)
library(dplyr)

Package version: 3.2.4
Unicode version: 13.0
ICU version: 66.1

Parallel computing: 4 of 4 threads used.

See https://quanteda.io for tutorials and examples.


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
# read data
cr_corpus <- readRDS("/paper-corpus-rds/paper_abstract_80prc.rds")

corp <- corpus(cr_corpus, text_field = "Abstract")

toks <- tokens(corp, 
               remove_punct = T, 
               remove_symbols = T, 
               remove_numbers = T, 
               remove_separators = T)

gc()

# clean out stop words and words with 2 or fewer characters
toks_nostop <- tokens_select(toks, 
                             pattern = stopwords("en"), 
                             selection = "remove", 
                             min_nchar = 2)
gc()

# only use features that appear at least 5 times in the corpus
feats <- dfm(toks_nostop, 
             tolower = T, 
             verbose = FALSE) %>% dfm_trim(min_termfreq = 5) %>% featnames()

gc()

# leave the pads so that non-adjacent words will not become adjacent
toks <- tokens_select(toks_nostop, feats, padding = TRUE)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,7463042,398.6,17221655,919.8,15370895,820.9
Vcells,193238978,1474.3,602333669,4595.5,553359735,4221.8


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,9049115,483.3,17221655,919.8,16475252,879.9
Vcells,261106274,1992.1,602333669,4595.5,553359735,4221.8


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,9061014,484.0,17221655,919.8,17221655,919.8
Vcells,261314552,1993.7,930092475,7096.1,1135333926,8662.0


In [4]:
#---------------------------------
# estimate glove model
#---------------------------------

# construct the feature co-occurrence matrix for our toks object (see above)
toks_fcm <- fcm(toks, 
                context = "window", 
                window = 6, 
                count = "frequency", 
                tri = FALSE)
gc()

# estimate glove model using text2vec
glove <- GlobalVectors$new(rank = 300, 
                           x_max = 10, 
                           learning_rate = 0.05)

wv_main <- glove$fit_transform(toks_fcm, 
                               n_iter = 1000, 
                               convergence_tol = 1e-3, 
                               n_threads = RcppParallel::defaultNumThreads())
gc()

wv_context <- glove$components
local_glove <- wv_main + t(wv_context)  # word vectors

gc()

saveRDS(local_glove, "/glove_paper_abstract_80prc.rds")

# qualitative check
# find_nns(local_glove["fairness", ], pre_trained = local_glove, N = 5, candidates = feats)


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,9061839,484.0,17221655,919.8,17221655,919.8
Vcells,427662084,3262.9,1669944291,12740.7,2086529069,15919.0


INFO  [06:03:47.964] epoch 1, loss 0.2152
INFO  [06:07:02.153] epoch 2, loss 0.1236
INFO  [06:10:15.922] epoch 3, loss 0.1093
INFO  [06:13:31.478] epoch 4, loss 0.1019
INFO  [06:16:46.284] epoch 5, loss 0.0970
INFO  [06:20:00.155] epoch 6, loss 0.0934
INFO  [06:23:15.096] epoch 7, loss 0.0905
INFO  [06:26:29.784] epoch 8, loss 0.0882
INFO  [06:29:43.698] epoch 9, loss 0.0862
INFO  [06:32:57.822] epoch 10, loss 0.0845
INFO  [06:36:11.488] epoch 11, loss 0.0829
INFO  [06:39:25.788] epoch 12, loss 0.0816
INFO  [06:42:39.296] epoch 13, loss 0.0804
INFO  [06:45:53.408] epoch 14, loss 0.0792
INFO  [06:49:06.825] epoch 15, loss 0.0782
INFO  [06:52:20.904] epoch 16, loss 0.0773
INFO  [06:55:34.911] epoch 17, loss 0.0764
INFO  [06:58:48.706] epoch 18, loss 0.0756
INFO  [07:02:03.014] epoch 19, loss 0.0749
INFO  [07:05:18.035] epoch 20, loss 0.0742
INFO  [07:08:31.889] epoch 21, loss 0.0735
INFO  [07:11:45.971] epoch 22, loss 0.0729
INFO  [07:14:59.556] epoch 23, loss 0.0723
INFO  [07:18:14.241]

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,9075523,484.7,17221655,919.8,17221655,919.8
Vcells,592555793,4520.9,1669944291,12740.7,2086529069,15919.0


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,9075563,484.7,17221655,919.8,17221655,919.8
Vcells,647389222,4939.2,1669944291,12740.7,2086529069,15919.0


In [5]:
# compute transform weighting = 'log' works well for smaller corpora for large
# corpora use a numeric value e.g. weighting = 500 see:
# https://arxiv.org/pdf/1805.05388.pdf
local_transform <- compute_transform(x = toks_fcm, pre_trained = local_glove, weighting = "log")


saveRDS(local_transform, "/local_transform_paper_abstract_80prc.rds")